# 8.1-1

8.1-1. The Childfair Company has three plants producing child push chairs that are to be shipped to four
distribution centers. Plants 1, 2, and 3 produce 12, 17, and 11 shipments per month, respectively. Each
distribution center needs to receive 10 shipments per month. The distance from each plant to the respective
distributing centers is given as shown below (in miles)

| Plant| Dist 1 | Dist 2 | Dist 3 | Dist 4 |
| --- | --- | --- | --- | --- |
| Plant 1  | 800 | 1300 | 400 | 700 |
| Plant 2  | 1100 | 1400 | 600 | 1000 |
| Plant 3  | 600 | 1200 | 800 | 900 |



The freight cost for each shipment is $100 plus 50 cents per mile. How much should be shipped from each plant to each
of the distribution centers to minimize the total shipping cost? (a) Formulate this problem as a transportation
problem by constructing the appropriate parameter table. (b) Draw the network representation of this problem. (c)
Obtain an optimal solution.

$Minimize: Z $



$$Z = \sum_{u,v \: \epsilon \:E} x_{uv}$$

$ 
s.t: 
$

Constraint 1: Ensures path starts at origin and that each subsequent edge in the path is a continuation from the previous edge. In other words, if the trip enters into a given city (excluding final destination), it must leave the same city on the next leg of the path. 
<br>
<br>
$$
\sum_{u,v \: \epsilon \:E} x_{uv} = 
\sum_{v,w \: \epsilon \:E} x_{uv} \qquad 
\forall  \enspace v \: \epsilon \:V
$$ 



Constraint 2: Ensures the path reaches the final destination. 
<br>
<br>
$$
\sum_{u,t \: \epsilon \:E} x_{ut} = 1 
$$

In [1]:
import pandas as pd
import numpy as np
from docplex.mp.model import Model
import docplex.mp.solution as Solution

In [2]:
# hard_coded
shipment_cost = 100
per_mile_cost = .50
max_demand = 10

# Initialize the problem data - external
df_distance = pd.read_csv('dat_ch8_1-1_distance.csv')
df_supply = pd.read_csv('dat_ch8_1-1_supply.csv')
df_demand = pd.read_csv('dat_ch8_1-1_demand.csv')

In [3]:
arcs = list((t.plant, t.distribution) for t in df_distance.itertuples())
plants = df_supply['plant']
distributions = df_demand['distribution']
distance = dict([((t.plant, t.distribution),t.distance ) for t in df_distance.itertuples()])
capacity = df_supply['capacity']
demand = df_demand['demand']


In [4]:
m = Model('8.1-1')

In [5]:
# Create decision variables
x = m.integer_var_dict(arcs, name ='x')
y = m.binary_var_dict(arcs, name = 'y')

In [6]:
# Objective function
m.minimize(m.sum(distance[ij]*per_mile_cost*x[ij] for ij in arcs)+m.sum(shipment_cost*x[ij] for ij in arcs))
print(m.export_to_string())

\ This file has been generated by DOcplex
\ ENCODING=ISO-8859-1
\Problem name: 8.1-1

Minimize
 obj: 500 x_1_1 + 750 x_1_2 + 300 x_1_3 + 450 x_1_4 + 650 x_2_1 + 800 x_2_2
      + 400 x_2_3 + 600 x_2_4 + 400 x_3_1 + 700 x_3_2 + 500 x_3_3 + 550 x_3_4
Subject To

Bounds
 0 <= y_1_1 <= 1
 0 <= y_1_2 <= 1
 0 <= y_1_3 <= 1
 0 <= y_1_4 <= 1
 0 <= y_2_1 <= 1
 0 <= y_2_2 <= 1
 0 <= y_2_3 <= 1
 0 <= y_2_4 <= 1
 0 <= y_3_1 <= 1
 0 <= y_3_2 <= 1
 0 <= y_3_3 <= 1
 0 <= y_3_4 <= 1

Binaries
 y_1_1 y_1_2 y_1_3 y_1_4 y_2_1 y_2_2 y_2_3 y_2_4 y_3_1 y_3_2 y_3_3 y_3_4

Generals
 x_1_1 x_1_2 x_1_3 x_1_4 x_2_1 x_2_2 x_2_3 x_2_4 x_3_1 x_3_2 x_3_3 x_3_4
End



In [7]:
plants

0    1
1    2
2    3
Name: plant, dtype: int64

In [8]:
for i in plants:
    print(i)
    print(capacity[i-1])


1
12
2
17
3
11


In [9]:
##### capacity constraint
for i in plants:
    m.add_constraint(m.sum(x[(i,j)]for j in distributions)<= capacity[i-1],ctname = 'capacity_plant%d'%i)
    

# demand constraint
for j in distributions:
    m.add_constraint(m.sum(x[i,j] for i in plants)>= demand[i-1], ctname = 'demand_dist%d'%j)

# # indicator constraint
# for i,j in arcs:
#     m.add_constraint(x[(i,j)] <= y[(i,j)]*max_demand, ctname = 'indicator_%d'%j)

# add_indicator
# for i,j in arcos:
#     mdl.add_indicator(x[(i,j)],d[i]+1==d[j], 
#                       name='order_(%d,_%d)'%(i, j))
                            
            

In [10]:
print(m.export_to_string())

\ This file has been generated by DOcplex
\ ENCODING=ISO-8859-1
\Problem name: 8.1-1

Minimize
 obj: 500 x_1_1 + 750 x_1_2 + 300 x_1_3 + 450 x_1_4 + 650 x_2_1 + 800 x_2_2
      + 400 x_2_3 + 600 x_2_4 + 400 x_3_1 + 700 x_3_2 + 500 x_3_3 + 550 x_3_4
Subject To
 capacity_plant1: x_1_1 + x_1_2 + x_1_3 + x_1_4 <= 12
 capacity_plant2: x_2_1 + x_2_2 + x_2_3 + x_2_4 <= 17
 capacity_plant3: x_3_1 + x_3_2 + x_3_3 + x_3_4 <= 11
 demand_dist1: x_1_1 + x_2_1 + x_3_1 >= 10
 demand_dist2: x_1_2 + x_2_2 + x_3_2 >= 10
 demand_dist3: x_1_3 + x_2_3 + x_3_3 >= 10
 demand_dist4: x_1_4 + x_2_4 + x_3_4 >= 10

Bounds
 0 <= y_1_1 <= 1
 0 <= y_1_2 <= 1
 0 <= y_1_3 <= 1
 0 <= y_1_4 <= 1
 0 <= y_2_1 <= 1
 0 <= y_2_2 <= 1
 0 <= y_2_3 <= 1
 0 <= y_2_4 <= 1
 0 <= y_3_1 <= 1
 0 <= y_3_2 <= 1
 0 <= y_3_3 <= 1
 0 <= y_3_4 <= 1

Binaries
 y_1_1 y_1_2 y_1_3 y_1_4 y_2_1 y_2_2 y_2_3 y_2_4 y_3_1 y_3_2 y_3_3 y_3_4

Generals
 x_1_1 x_1_2 x_1_3 x_1_4 x_2_1 x_2_2 x_2_3 x_2_4 x_3_1 x_3_2 x_3_3 x_3_4
End



In [11]:
m.parameters.timelimit=120
m.parameters.mip.strategy.branch=1
m.parameters.mip.tolerances.mipgap=0.15

soln = m.solve(log_output=True)

Version identifier: 12.10.0.0 | 2019-11-27 | 843d4de
CPXPARAM_Read_DataCheck                          1
CPXPARAM_RandomSeed                              201903125
CPXPARAM_MIP_Strategy_Branch                     1
CPXPARAM_TimeLimit                               120
CPXPARAM_MIP_Tolerances_MIPGap                   0.14999999999999999
Found incumbent of value 22500.000000 after 0.00 sec. (0.00 ticks)
Tried aggregator 1 time.
MIP Presolve eliminated 0 rows and 12 columns.
Reduced MIP has 7 rows, 12 columns, and 24 nonzeros.
Reduced MIP has 0 binaries, 12 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.02 ticks)
Tried aggregator 1 time.
Detecting symmetries...
Reduced MIP has 7 rows, 12 columns, and 24 nonzeros.
Reduced MIP has 0 binaries, 12 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.01 ticks)
MIP emphasis: balance optimality and feasibility.
MIP search method: dynamic search.
Parallel mode: deterministic, using up to 12 threads.
Root relaxation sol

In [12]:
m.get_solve_status()


<JobSolveStatus.OPTIMAL_SOLUTION: 2>

In [13]:
soln.display()

solution for: 8.1-1
objective: 20200.000
x_1_3 = 2
x_1_4 = 10
x_2_2 = 9
x_2_3 = 8
x_3_1 = 10
x_3_2 = 1


In [25]:
lst = []
for i in x:
    if soln.get_var_value(x[i]) >0:
        solution =(i[0],i[1],soln.get_var_value(x[i]))
        lst.append(solution)
df = pd.DataFrame.from_records(lst,columns = ['plant','distribution','quantity'])


[(1, 3, 2.0),
 (1, 4, 10.0),
 (2, 2, 9.0),
 (2, 3, 8.0),
 (3, 1, 10.0),
 (3, 2, 1.0)]

In [27]:

df

,plant,distribution,quantity
0,1,3,2.0
1,1,4,10.0
2,2,2,9.0
3,2,3,8.0
4,3,1,10.0
5,3,2,1.0


In [30]:
df.to_csv('dat_ch8_1-1_solution.csv', index = False)

In [ ]:
lst = []
for i in cities:
#     if soln.get_var_value(d[i]) >0:
    print(d[i], soln.get_var_value(d[i]))

In [ ]:
soln.get_var_value(c[0])